# Datathon — Pré-processamento e Construção do Dataset de Treino

Este notebook implementa o pipeline de preparação de dados para o modelo de alerta preventivo de risco educacional. O objetivo é transformar os dados brutos do PEDE em datasets de treino e validação prontos para a etapa de engenharia de features.

---

## Decisões de design

| Decisão | Justificativa |
|---|---|
| **Formulação do target como piora de condição** (`defasagem_t1 > defasagem_t`) | Inclui todos os alunos elegíveis (não apenas os em dia), ampliando o conjunto de treino sem comprometer o rigor metodológico |
| **Split temporal** (treino: 2022→2023 / validação: 2023→2024) | Respeita a ordem cronológica — evita data leakage entre partições |
| **Anti-leakage** (remoção de `INDE`, `Pedra`, `IPV`, `IAN`, `IDA`, `Cg/Cf/Ct`, `Atingiu PV`, recomendações) | Elimina proxies do resultado institucional que não estariam disponíveis no momento da predição |
| **Inglês neutralizado por fase** (`ALFA`, `FASE1`, `FASE2`, `FASE8`) | Zera o campo onde inglês não é obrigatório, evitando viés sistemático por ausência |
| **Política operacional Top-K% estratificada por Fase** | Garante representatividade de todas as fases na lista de alertas, independente do tamanho de cada grupo |

## Métrica principal

> **Recall@TopK** — fração dos alunos em risco real que o modelo consegue identificar dentro do orçamento operacional K%.  
> AUC, Precision@K e Lift@K são métricas de apoio para diagnóstico do modelo.

## Saídas deste notebook

- `notebooks/data/train_df.parquet` — dataset de treino (par 2022→2023)
- `notebooks/data/valid_df.parquet` — dataset de validação (par 2023→2024)


In [53]:
from pathlib import Path
import re
import numpy as np
import pandas as pd

# (Opcional) logging simples sem muita estrutura
import logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s | %(message)s")
log = logging.getLogger("PIPELINE")

# Caminhos locais (relativo à raiz do workspace)
ROOT_DIR = Path("..").resolve()  # notebooks/ -> workspace root
DATA_DIR = ROOT_DIR / "data"
NOTEBOOKS_DIR = ROOT_DIR / "notebooks"

XLS_PATH = DATA_DIR / "BASE DE DADOS PEDE 2024 - DATATHON.xlsx"
assert XLS_PATH.exists(), f"Arquivo não encontrado: {XLS_PATH}"
print("✅ XLS:", XLS_PATH)


✅ XLS: /home/glauberthy/Desktop/datathon/data/BASE DE DADOS PEDE 2024 - DATATHON.xlsx


## 1. Coleta de Dados — Ingestão do XLS

O arquivo fonte é o **PEDE** (*Pesquisa Extensiva do Desenvolvimento do Estudante*), uma planilha multi-ano com uma aba por ciclo (`PEDE2022`, `PEDE2023`, `PEDE2024`).

**Estratégia:** detectar automaticamente todas as abas com padrão `PEDEYYYY` e carregá-las em um dicionário `{ano: DataFrame}`, sem hardcodar os anos disponíveis.

> **Por quê multi-ano?** O modelo aprende *transições* entre anos consecutivos (T → T+1). Quanto mais pares históricos disponíveis, maior a capacidade de generalização — especialmente para fases com poucos alunos.


In [54]:
# Ler abas PEDEYYYY
xls = pd.ExcelFile(XLS_PATH)
sheet_names = xls.sheet_names

year_sheets = []
for s in sheet_names:
    m = re.search(r"(?:PEDE\s*)?(20\d{2})", s, flags=re.IGNORECASE)
    if m:
        year_sheets.append((int(m.group(1)), s))

year_sheets = sorted(set(year_sheets))
log.info("Abas/anos detectados: %s", year_sheets)

data_raw = {}
for year, sheet in year_sheets:
    df = pd.read_excel(XLS_PATH, sheet_name=sheet)
    df["ano_base"] = year
    data_raw[year] = df

list(data_raw.keys()), {y: data_raw[y].shape for y in data_raw}


INFO | Abas/anos detectados: [(2022, 'PEDE2022'), (2023, 'PEDE2023'), (2024, 'PEDE2024')]


([2022, 2023, 2024], {2022: (860, 43), 2023: (1014, 49), 2024: (1156, 51)})

*Checagem rápida*

In [55]:
print(data_raw[2022].columns, data_raw[2023].columns, data_raw[2024].columns)

Index(['RA', 'Fase', 'Turma', 'Nome', 'Ano nasc', 'Idade 22', 'Gênero',
       'Ano ingresso', 'Instituição de ensino', 'Pedra 20', 'Pedra 21',
       'Pedra 22', 'INDE 22', 'Cg', 'Cf', 'Ct', 'Nº Av', 'Avaliador1',
       'Rec Av1', 'Avaliador2', 'Rec Av2', 'Avaliador3', 'Rec Av3',
       'Avaliador4', 'Rec Av4', 'IAA', 'IEG', 'IPS', 'Rec Psicologia', 'IDA',
       'Matem', 'Portug', 'Inglês', 'Indicado', 'Atingiu PV', 'IPV', 'IAN',
       'Fase ideal', 'Defas', 'Destaque IEG', 'Destaque IDA', 'Destaque IPV',
       'ano_base'],
      dtype='object') Index(['RA', 'Fase', 'INDE 2023', 'Pedra 2023', 'Turma', 'Nome Anonimizado',
       'Data de Nasc', 'Idade', 'Gênero', 'Ano ingresso',
       'Instituição de ensino', 'Pedra 20', 'Pedra 21', 'Pedra 22', 'Pedra 23',
       'INDE 22', 'INDE 23', 'Cg', 'Cf', 'Ct', 'Nº Av', 'Avaliador1',
       'Rec Av1', 'Avaliador2', 'Rec Av2', 'Avaliador3', 'Rec Av3',
       'Avaliador4', 'Rec Av4', 'IAA', 'IEG', 'IPS', 'IPP', 'Rec Psicologia',
       'IDA'

## 2. Pré-processamento — Padronização e Anti-leakage

O PEDE não tem schema fixo entre os anos: nomes de colunas variam em grafia, acentuação e abreviação. Esta seção resolve isso em três etapas obrigatórias antes de qualquer junção longitudinal:

1. **Normalização de nomes de colunas** — garante que a mesma variável tenha o mesmo nome em todos os anos
2. **Renomeação canônica** — mapeia aliases históricos para um nome único (ex.: `mat` → `matem`, `ing` → `ingles`)
3. **Anti-leakage** — remove variáveis que só existem *após* a decisão institucional (`INDE`, `Pedra`, `IPV`, `IAN`, `IDA`, `Cg/Cf/Ct`, `Atingiu PV`, recomendações)

> **Por quê o anti-leakage importa?** Variáveis como `INDE` e `Pedra` são calculadas *depois* que a situação do aluno já é conhecida. Incluí-las seria equivalente a prever o futuro usando o futuro — o modelo teria AUC artificial próximo de 1.0 em treino, mas falharia completamente em produção.


### 2.1 Normalização de Nomes de Colunas

Transforma os cabeçalhos brutos do Excel em identificadores limpos e consistentes entre os anos:

| Transformação | Exemplo |
|---|---|
| Remove acentos e caracteres especiais | `Instituição de ensino` → `Instituicao de ensino` |
| Converte para minúsculas | `Matematica` → `matematica` |
| Substitui qualquer caractere fora de `[a-z0-9]` por `_` | `fase ideal` → `fase_ideal` |
| Remove `_` nas extremidades | `_fase_` → `fase` |


In [56]:
import unicodedata

def normalizar_nome_colunas(col: str) -> str:
    s = unicodedata.normalize("NFKD", str(col)).encode("ascii","ignore").decode("ascii")
    s = s.lower().strip()
    s = re.sub(r"[^a-z0-9]+", "_", s).strip("_")
    return s

# Normaliza nomes de colunas para facilitar matching (SEM perder colunas)
data_raw[2022].columns = [normalizar_nome_colunas(c) for c in data_raw[2022].columns]
data_raw[2023].columns = [normalizar_nome_colunas(c) for c in data_raw[2023].columns]
data_raw[2024].columns = [normalizar_nome_colunas(c) for c in data_raw[2024].columns]

*Checagem rápida*

In [57]:
list(data_raw[2022].columns)

['ra',
 'fase',
 'turma',
 'nome',
 'ano_nasc',
 'idade_22',
 'genero',
 'ano_ingresso',
 'instituicao_de_ensino',
 'pedra_20',
 'pedra_21',
 'pedra_22',
 'inde_22',
 'cg',
 'cf',
 'ct',
 'no_av',
 'avaliador1',
 'rec_av1',
 'avaliador2',
 'rec_av2',
 'avaliador3',
 'rec_av3',
 'avaliador4',
 'rec_av4',
 'iaa',
 'ieg',
 'ips',
 'rec_psicologia',
 'ida',
 'matem',
 'portug',
 'ingles',
 'indicado',
 'atingiu_pv',
 'ipv',
 'ian',
 'fase_ideal',
 'defas',
 'destaque_ieg',
 'destaque_ida',
 'destaque_ipv',
 'ano_base']

In [58]:
list(data_raw[2022].columns) == list(data_raw[2023].columns) == list(data_raw[2024].columns)

False

### 2.2 Renomeação Canônica

O PEDE usa abreviações inconsistentes entre anos para a mesma variável. Esta etapa consolida todos os aliases num nome canônico único:

| Aliases no fonte | Nome canônico |
|---|---|
| `mat`, `matem` | `matem` |
| `por`, `portug` | `portug` |
| `ing`, `ingles` | `ingles` |
| `instituicao_de_ensino` | `instituicao` |
| `fase` | `fase_raw` (preserva o valor original antes da normalização numérica) |
| `defas` | `defasagem` |
| `cf`, `cg`, `ct` | `class_fase`, `class_geral`, `class_turma` |
| `inde_22` / `inde_23` / `inde_24` | `inde_2022` / `inde_2023` / `inde_2024` |

> Colunas sem alias conhecido são mantidas com o nome que saíram da etapa anterior — nenhuma coluna é descartada aqui.


In [59]:
def apply_canonical_renames(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    rename_candidates = {
        # identificadores
        "ra": "ra",
        "turma": "turma",
        "nome":"nome",
        "nome_anonimizado":"nome",
        "genero": "genero",
        "instituicao_de_ensino": "instituicao",
        "ano_ingresso": "ano_ingresso",
        "fase": "fase_raw",
        "fase_raw": "fase_raw",
        # provas
        "matem": "matem",
        "mat":"matem",
        "por":"portug",
        "portug": "portug",
        "ingles": "ingles",
        "ing":"ingles",
        # indicadores
        "ieg": "ieg",
        "iaa": "iaa",
        "ips": "ips",
        "ipp": "ipp",
        "no_av": "no_av",
        # defasagem
        "defas":"defasagem",
        # nascimento (se existir)
        "idade": "idade",
        "ano_nasc": "ano_nasc",
        "data_de_nasc": "data_nasc",
        #outros
        'inde_22': 'inde_2022', 'inde_22': 'inde_2022',
        'inde_23': 'inde_2023', 'inde_23': 'inde_2023',
        'inde_24': 'inde_2024', 'inde_24': 'inde_2024',
        'pedra_21': 'pedra_2021',
        'pedra_20': 'pedra_2020',
        'pedra_2022': 'pedra_2022', 'pedra_22': 'pedra_2022',
        'pedra_2023': 'pedra_2023', 'pedra_23': 'pedra_2023',
        'pedra_2024':'pedra_2024', 'pedra_24': 'pedra_2024',
        'cf': 'class_fase', 'cg': 'class_geral', 'ct': 'class_turma',
    }

    df = df.rename(columns={c: rename_candidates[c] for c in df.columns if c in rename_candidates})
    return df

### 2.3 Coerção de Tipos e Limpeza de Valores

Após a renomeação canônica, cada coluna recebe o tipo correto e os valores inconsistentes são tratados:

| Campo | Tratamento |
|---|---|
| `ra` | String, sem espaços laterais |
| `fase` | Cópia de `fase_raw` normalizada para uppercase; `NaN` onde vazio |
| `data_nasc` | Convertida para `datetime` com `dayfirst=True` |
| `ano_nasc` | `Int64` nullable; derivado de `data_nasc` quando ausente |
| `idade` | `Int64` nullable; recalculada como `ano_base - ano_nasc`; descartada se fora de [0, 120] |
| Numéricos (`defasagem`, `matem`, `portug`, `ingles`, `ieg`, `iaa`, `ips`, `ipp`, `ano_ingresso`) | `pd.to_numeric(..., errors="coerce")` — valores não parseáveis viram `NaN` |
| `turma`, `genero`, `instituicao` | String sem espaços; valores `"NAN"` / `""` substituídos por `NaN` |

> Colunas obrigatórias ausentes num dado ano são preenchidas com `NaN` para garantir schema uniforme antes do concat longitudinal.


In [60]:
def coerce_types_and_clean(df: pd.DataFrame, year: int) -> pd.DataFrame:
    df = df.copy()
    df["ano_base"] = int(year)

    # garante mínimas (compatibilidade do pipeline)
    must_have = [
        "ra","ano_base","fase_raw","turma","defasagem",
        "matem","portug","ingles","ieg","iaa","ips","ipp",
        "genero","instituicao","ano_ingresso"
    ]
    for c in must_have:
        if c not in df.columns:
            df[c] = np.nan

    # RA / fase
    df["ra"] = df["ra"].astype(str).str.strip()
    df["fase"] = (
        df["fase_raw"].astype(str).str.strip().str.upper()
        .replace({"NAN": np.nan, "nan": np.nan, "": np.nan})
    )

    # coerção datas (se existir)
    if "data_nasc" in df.columns:
        df["data_nasc"] = pd.to_datetime(df["data_nasc"], errors="coerce", dayfirst=True)

    # Derivar ano_nasc. Priorizar 'ano_nasc' existente, depois derivar de 'data_nasc'.
    if "ano_nasc" in df.columns:
        df["ano_nasc"] = pd.to_numeric(df["ano_nasc"], errors="coerce").astype("Int64")
    else:
        df["ano_nasc"] = pd.NA

    if "data_nasc" in df.columns:
        df.loc[df["data_nasc"].notna(), "ano_nasc"] = df.loc[df["data_nasc"].notna(), "data_nasc"].dt.year.astype("Int64")

    # Sempre recalcular idade se ano_nasc estiver disponível e for sensato
    if "ano_nasc" in df.columns and not df["ano_nasc"].isna().all():
        ref_year = df["ano_base"].astype(int)
        idade = (ref_year - df["ano_nasc"]).astype("Int64")
        idade = idade.where((idade >= 0) & (idade <= 120), pd.NA)
        df["idade"] = idade
    else:
        df["idade"] = pd.NA

    # Coerção numérica para outras colunas
    other_num_cols = [c for c in ["defasagem","matem","portug","ingles","ieg","iaa","ips","ipp","ano_ingresso","no_av"] if c not in ["ano_nasc", "idade"]]
    for c in other_num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # sanitização strings
    if "turma" in df.columns:
        df["turma"] = df["turma"].astype(str).str.strip().replace({"NAN": np.nan, "nan": np.nan, "": np.nan})
    if "genero" in df.columns:
        df["genero"] = (
            df["genero"].astype(str).str.strip().str.upper()
            .replace({"NAN": np.nan, "nan": np.nan, "": np.nan})
        )
    if "instituicao" in df.columns:
        df["instituicao"] = (
            df["instituicao"].astype(str).str.strip()
            .replace({"NAN": np.nan, "nan": np.nan, "": np.nan})
        )

    # reordena (canônicas primeiro) — mas NÃO remove as outras
    front = [c for c in must_have + ["fase","data_nasc","ano_nasc","idade","no_av"] if c in df.columns]
    other_cols = [c for c in df.columns if c not in front]
    df = df[front + other_cols]

    return df


def standardize_year(df: pd.DataFrame, year: int) -> pd.DataFrame:
    df = apply_canonical_renames(df)
    df = coerce_types_and_clean(df, year=year)
    return df


data = {y: standardize_year(data_raw[y], y) for y in sorted(data_raw)}


/tmp/ipykernel_193383/1837260891.py:24: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["data_nasc"] = pd.to_datetime(df["data_nasc"], errors="coerce", dayfirst=True)


### 2.4 Orquestração — Padronização por Ano

`standardize_year` encadeia as duas etapas anteriores em uma única chamada por ano:

```
data_raw[y]  →  apply_canonical_renames  →  coerce_types_and_clean  →  data[y]
```

O dicionário resultante `data = {ano: DataFrame}` é a entrada de todas as etapas seguintes.


In [61]:
data[2022]['fase'].unique(), data[2023]['fase'].unique(), data[2024]['fase'].unique()

(array(['7', '6', '5', '4', '3', '2', '1', '0'], dtype=object),
 array(['ALFA', 'FASE 1', 'FASE 2', 'FASE 3', 'FASE 4', 'FASE 5', 'FASE 6',
        'FASE 7', 'FASE 8'], dtype=object),
 array(['ALFA', '1A', '1B', '1C', '1D', '1E', '1G', '1H', '1J', '1K', '1L',
        '1M', '1N', '1P', '1R', '2A', '2B', '2C', '2D', '2G', '2H', '2I',
        '2K', '2L', '2M', '2N', '2P', '2R', '2U', '3A', '3B', '3C', '3D',
        '3F', '3G', '3H', '3I', '3K', '3L', '3M', '3N', '3P', '3R', '3U',
        '4A', '4B', '4C', '4F', '4H', '4L', '4M', '4N', '4R', '5A', '5B',
        '5C', '5D', '5F', '5G', '5L', '5M', '5N', '6A', '6L', '7A', '7E',
        '8A', '8B', '8D', '8E', '8F', '9'], dtype=object))

### 2.5 Consolidação de INDE e Pedra

O PEDE armazena o INDE e a Pedra em colunas separadas por ano (`inde_2022`, `inde_23`, etc.). Esta etapa cria colunas canônicas `inde` e `pedra` que contêm apenas os valores do **próprio ano base** de cada DataFrame:

| Risco sem este passo | Consequência |
|---|---|
| Usar `inde_2023` em `data[2022]` | Leakage — informação futura vazando para o passado |
| Misturar colunas de anos diferentes | Valores de `pedra` de outro ciclo contaminando features do ano errado |

> A detecção do ano é automática: a função inspeciona `data["ano_base"].iloc[0]` e localiza a coluna `inde_{ano}` correspondente, sem hardcoding.


In [62]:
def ensure_inde_pedra(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    ano = df["ano_base"].iloc[0]
    if pd.isna(ano):
        df["inde"] = pd.NA
        df["pedra"] = pd.NA
        return df

    ano = int(ano)
    ano2 = str(ano)[-2:]

    # nomes válidos EXCLUSIVOS do ano base
    inde_cols = [f"inde_{ano}", f"inde_{ano2}"]
    pedra_cols = [f"pedra_{ano}", f"pedra_{ano2}"]

    def _first_series(df: pd.DataFrame, col: str) -> pd.Series:
        """Retorna a primeira ocorrência da coluna como Series (evita erro com colunas duplicadas)."""
        col_data = df[col]
        if isinstance(col_data, pd.DataFrame):
            return col_data.iloc[:, 0]
        return col_data

    inde_col = next((c for c in inde_cols if c in df.columns), None)
    pedra_col = next((c for c in pedra_cols if c in df.columns), None)

    df["inde"] = _first_series(df, inde_col) if inde_col else pd.NA
    df["pedra"] = _first_series(df, pedra_col) if pedra_col else pd.NA

    return df

data[2022] = ensure_inde_pedra(data[2022])
data[2023] = ensure_inde_pedra(data[2023])
data[2024] = ensure_inde_pedra(data[2024])

print('data[2022]["inde"]:\n', data[2022]["inde"].head(2))
print('\ndata[2023]["inde"]:\n', data[2023]["inde"].head(2))
print('\ndata[2024]["inde"]:\n', data[2024]["inde"].head(2))

print('\ndata[2022]["pedra"]:\n', data[2022]["pedra"].head(3))
print('\ndata[2023]["pedra"]:\n', data[2023]["pedra"].head(3))
print('\ndata[2024]["pedra"]:\n', data[2024]["pedra"].head(3))

for name, df_obj in [('df2023', data[2023]), ('df2024', data[2024])]:
    print(name, "inde nulos %:", float(df_obj["inde"].isna().mean()*100), "| pedra nulos %:", float(df_obj["pedra"].isna().mean()*100))


data[2022]["inde"]:
 0    5.783
1    7.055
Name: inde, dtype: float64

data[2023]["inde"]:
 0    9.31095
1    8.22120
Name: inde, dtype: float64

data[2024]["inde"]:
 0    7.611367
1    8.002867
Name: inde, dtype: object

data[2022]["pedra"]:
 0     Quartzo
1    Ametista
2       Ágata
Name: pedra, dtype: object

data[2023]["pedra"]:
 0    Topázio
1    Topázio
2    Quartzo
Name: pedra, dtype: object

data[2024]["pedra"]:
 0    Ametista
1     Topázio
2    Ametista
Name: pedra, dtype: object
df2023 inde nulos %: 8.185404339250493 | pedra nulos %: 8.185404339250493
df2024 inde nulos %: 5.536332179930796 | pedra nulos %: 5.536332179930796


### 2.6 Remoção de Colunas Anuais Redundantes

Após a consolidação em `inde` e `pedra`, as colunas originais com sufixo de ano (`inde_2022`, `pedra_23`, etc.) são removidas. Isso evita que o modelo acesse acidentalmente valores de INDE/Pedra de outros ciclos como features.

> O padrão de remoção é `(inde|pedra)_(\d{2}|\d{4})` — cobre tanto sufixos com 2 dígitos quanto com 4 dígitos.


In [63]:
def drop_only_annual_inde_pedra(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    drop_cols = [c for c in df.columns if re.fullmatch(r"(inde|pedra)_(\d{2}|\d{4})", c)]
    return df.drop(columns=drop_cols, errors="ignore")

data[2022] = drop_only_annual_inde_pedra(data[2022])
data[2023] = drop_only_annual_inde_pedra(data[2023])
data[2024] = drop_only_annual_inde_pedra(data[2024])

print('\ndata[2023]["data_nasc"]:\n', data[2023]["data_nasc"].head(2))
print('\ndata[2024]["data_nasc"]:\n', data[2024]["data_nasc"].head(2))


data[2023]["data_nasc"]:
 0   2015-06-17
1   2014-05-31
Name: data_nasc, dtype: datetime64[ns]

data[2024]["data_nasc"]:
 0   2016-07-28
1   2016-10-16
Name: data_nasc, dtype: datetime64[ns]


In [64]:
data[2022].shape, data[2023].shape, data[2024].shape

((860, 44), (1014, 45), (1156, 47))

### 2.7 Normalização de Valores Categóricos

Padroniza os valores textuais de `genero`, `pedra` e `fase` para um vocabulário canônico controlado:

| Campo | Valores de entrada (exemplos) | Valor canônico |
|---|---|---|
| `genero` | `"menino"`, `"masculino"` | `"M"` |
| `genero` | `"menina"`, `"feminino"` | `"F"` |
| `pedra` | `"quartzo"`, `"ametista"`, `"topazio"`, `"agata"` | mantido em minúsculas |
| `pedra` | `"incluir"` (fases 8–9) | `"sem_pedra_fase_8_9"` |
| `fase` | `"ALFA"` | `0` (`Int64`) |
| `fase` | `"FASE 3"`, `"3"` | `3` (`Int64`) |

Valores não reconhecidos viram `NaN` — o modelo não recebe strings inesperadas.


In [65]:
data[2022]['genero'].unique(), data[2023]['genero'].unique(), data[2024]['genero'].unique()

(array(['MENINA', 'MENINO'], dtype=object),
 array(['FEMININO', 'MASCULINO'], dtype=object),
 array(['MASCULINO', 'FEMININO'], dtype=object))

In [66]:
data[2022]['pedra'].unique(), data[2023]['pedra'].unique(), data[2024]['pedra'].unique()

(array(['Quartzo', 'Ametista', 'Ágata', 'Topázio'], dtype=object),
 array(['Topázio', 'Quartzo', 'Ametista', 'Agata', nan], dtype=object),
 array(['Ametista', 'Topázio', 'Quartzo', 'Agata', nan, 'INCLUIR'],
       dtype=object))

In [67]:
import pandas as pd
import numpy as np
import re
import unicodedata

def _strip_accents(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s))
    return "".join(ch for ch in s if not unicodedata.combining(ch))

def normalize_colname(col: str) -> str:
    col = _strip_accents(col).strip().lower()
    col = re.sub(r"[\/\-\.\,\(\)\[\]\{\}]+", " ", col)
    col = re.sub(r"\s+", "_", col)
    col = re.sub(r"[^a-z0-9_]", "", col)
    col = re.sub(r"_+", "_", col).strip("_")
    return col

GENERO_MAP = {
    "menino": "M",
    "masculino": "M",
    "menina": "F",
    "feminino": "F",
}

PEDRA_VALID = {
    "quartzo": "quartzo",
    "ametista": "ametista",
    "topazio": "topazio",
    "agata": "agata",
    "incluir": "sem_pedra_fase_8_9",
}

def normalize_pedra_value(x):
    if pd.isna(x):
        return pd.NA
    s = normalize_colname(x)
    if s == "":
        return pd.NA
    return PEDRA_VALID.get(s, pd.NA)

def normalize_fase(value):
    if pd.isna(value):
        return pd.NA
    v = str(value).strip().upper()
    if v == "ALFA":
        return 0
    match = re.search(r"\d+", v)
    if match:
        fase = int(match.group())
        if 0 <= fase <= 9:
            return fase
    return pd.NA

def normalize_genero_value(x):
    if pd.isna(x):
        return pd.NA
    s = normalize_colname(x)
    return GENERO_MAP.get(s, pd.NA)

def normalize_text_fields(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    if "genero" in df.columns:
        df["genero"] = df["genero"].apply(normalize_genero_value)

    if "pedra" in df.columns:
        df["pedra"] = df["pedra"].apply(normalize_pedra_value)

    if "fase" in df.columns:
        df["fase"] = df["fase"].apply(normalize_fase).astype("Int64")

    return df

for year in [2022, 2023, 2024]:
    if year in data:
        data[year] = normalize_text_fields(data[year])

In [68]:
data[2022]['fase_raw'].unique(), data[2023]['fase_raw'].unique(), data[2024]['fase_raw'].unique()

(array([7, 6, 5, 4, 3, 2, 1, 0]),
 array(['ALFA', 'FASE 1', 'FASE 2', 'FASE 3', 'FASE 4', 'FASE 5', 'FASE 6',
        'FASE 7', 'FASE 8'], dtype=object),
 array(['ALFA', '1A', '1B', '1C', '1D', '1E', '1G', '1H', '1J', '1K', '1L',
        '1M', '1N', '1P', '1R', '2A', '2B', '2C', '2D', '2G', '2H', '2I',
        '2K', '2L', '2M', '2N', '2P', '2R', '2U', '3A', '3B', '3C', '3D',
        '3F', '3G', '3H', '3I', '3K', '3L', '3M', '3N', '3P', '3R', '3U',
        '4A', '4B', '4C', '4F', '4H', '4L', '4M', '4N', '4R', '5A', '5B',
        '5C', '5D', '5F', '5G', '5L', '5M', '5N', '6A', '6L', '7A', '7E',
        '8A', '8B', '8D', '8E', '8F', 9], dtype=object))

In [69]:
data[2022]['fase'].unique(), data[2023]['fase'].unique(), data[2024]['fase'].unique()

(<IntegerArray>
 [7, 6, 5, 4, 3, 2, 1, 0]
 Length: 8, dtype: Int64,
 <IntegerArray>
 [0, 1, 2, 3, 4, 5, 6, 7, 8]
 Length: 9, dtype: Int64,
 <IntegerArray>
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
 Length: 10, dtype: Int64)

In [70]:
data[2022]['genero'].unique(), data[2023]['genero'].unique(), data[2024]['genero'].unique()

(array(['F', 'M'], dtype=object),
 array(['F', 'M'], dtype=object),
 array(['M', 'F'], dtype=object))

In [71]:
data[2022]['pedra'].unique(), data[2023]['pedra'].unique(), data[2024]['pedra'].unique()

(array(['quartzo', 'ametista', 'agata', 'topazio'], dtype=object),
 array(['topazio', 'quartzo', 'ametista', 'agata', <NA>], dtype=object),
 array(['ametista', 'topazio', 'quartzo', 'agata', <NA>,
        'sem_pedra_fase_8_9'], dtype=object))

In [72]:
data[2022].shape, data[2023].shape, data[2024].shape

((860, 44), (1014, 45), (1156, 47))

### 2.8 Normalização Final de Tipos

Garante que cada coluna tenha o dtype correto antes da modelagem. Vírgulas decimais (`","`) são convertidas para ponto (`"."`) e strings não numéricas viram `NaN`:

| Grupo | Colunas | Dtype alvo |
|---|---|---|
| Identificador | `ra` | `string` |
| Inteiros nullable | `idade`, `ano_ingresso`, `fase`, `no_av`, `defasagem`, `ano_nasc` | `Int64` |
| Floats | `ieg`, `iaa`, `ips`, `ida`, `ipv`, `matem`, `portug`, `ingles` | `float64` |
| Categorias | `genero`, `turma` | `category` |

> Avisos são emitidos quando valores viram `NaN` após conversão, permitindo auditar a qualidade dos dados por coluna.


In [73]:
#Normalizando os tipos de dados

def normalize_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # id
    if "ra" in df.columns:
        df["ra"] = df["ra"].astype("string").str.strip()

    # ano_nasc (quando existir)
    # Convert to numeric, handle potential non-integer floats by rounding, then cast to nullable integer
    numeric_ano_nascimento = pd.to_numeric(df["ano_nasc"], errors="coerce")
    df["ano_nasc"] = numeric_ano_nascimento.apply(lambda x: round(x) if pd.notna(x) else pd.NA).astype("Int64")

    # Preenche ano_nasc somente onde estiver nulo
    # Ensure data_nascimento.dt.year is also treated as integer (it usually is)
    if "data_nasc" in df.columns and pd.api.types.is_datetime64_any_dtype(df["data_nasc"]):
        df["ano_nasc"] = df["ano_nasc"].fillna(df["data_nasc"].dt.year.astype("Int64"))
    else:
        # If data_nascimento is not datetime or doesn't exist, fill with NA
        df["ano_nasc"] = df["ano_nasc"].fillna(pd.NA)

    # inteiros
    for col in ["idade", "ano_ingresso", "fase", "no_av", "defasagem"]:
        if col in df.columns:
            numeric_series = pd.to_numeric(df[col], errors="coerce")
            df[col] = numeric_series.apply(lambda x: round(x) if pd.notna(x) else pd.NA).astype("Int64")

    # floats
    for col in ["ida", "ieg", "iaa", "ips", "ipv", "matam", "portug", "ingles"]:
      if col in df.columns:
          before_na = df[col].isna().sum()

          df[col] = (
              df[col]
              .astype(str)
              .str.replace(",", ".", regex=False)
              .str.strip()
         )
          df[col] = pd.to_numeric(df[col], errors="coerce")

          after_na = df[col].isna().sum()
          if after_na > before_na:
             print(f"⚠️ {col}: {after_na - before_na} valores viraram NaN após conversão")

    # categorias (depois de normalizar valores!)
    for col in ["genero", "turma"]:
        if col in df.columns:
            df[col] = df[col].astype("category")

    return df

data[2022] = normalize_dtypes(data[2022])
data[2023] = normalize_dtypes(data[2023])
data[2024] = normalize_dtypes(data[2024])


In [74]:
data[2022].shape, data[2023].shape, data[2024].shape

((860, 44), (1014, 45), (1156, 47))

In [75]:
nan_count_2022 = data[2022]['defasagem'].isna().sum()
nan_count_2023 = data[2023]['defasagem'].isna().sum()
nan_count_2024 = data[2024]['defasagem'].isna().sum()

print(f"Número de NaN no campo 'defasagem' 2022, 2023, 2024 é: {nan_count_2022} / {nan_count_2023} / {nan_count_2024}")

Número de NaN no campo 'defasagem' 2022, 2023, 2024 é: 0 / 0 / 0


## 3. Modelo de Dados — Fatos, Dimensões e Views

Organiza os DataFrames padronizados em uma estrutura estrela antes de montar os pares longitudinais:

| Tabela | Granularidade | Conteúdo | Uso |
|---|---|---|---|
| `fact_student_year` | `ra` + `ano_base` | Notas, indicadores psicossociais, `defasagem` | Base do modelo |
| `dim_student` | `ra` | Gênero, nascimento, ingresso, nome | Join para features demográficas |
| `dim_indicators_year` | `ra` + `ano_base` | `inde`, `pedra`, `ida`, `ian`, `ipv`, classificações | BI / auditoria — **não entra no modelo** |
| `dim_assessment_process` | `ra` + `ano_base` | Avaliadores, recomendações, `atingiu_pv`, destaques | Auditoria operacional |
| `vw_ml_train_ready_star` | `ra` + `ano_base` | `fact` ⋈ demográficos do `dim_student` | Input do pipeline longitudinal |
| `vw_dashboard_360` | `ra` + `ano_base` | `fact` ⋈ indicadores ⋈ processo | Visão 360 para coordenadores |
| `vw_audit_assessment` | `ra` + `ano_base` | Processo ⋈ fact (notas / alvo) | Detecção de vieses operacionais |

> `dim_indicators_year` e `dim_assessment_process` são excluídos do treino: contêm variáveis calculadas após o encerramento do ciclo (leakage potencial).


In [76]:
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 0) Unir anos em um DF único (se ainda não tiver df_all)
# ---------------------------------------------------------
df_all = pd.concat([data[y] for y in sorted(data)], ignore_index=True)

In [77]:
# Garantias básicas
assert {"ra","ano_base"}.issubset(df_all.columns), "Faltam colunas ra/ano_base no df_all."
df_all["ra"] = df_all["ra"].astype(str).str.strip()
df_all["ano_base"] = pd.to_numeric(df_all["ano_base"], errors="coerce").astype("Int64")

In [78]:
# ---------------------------------------------------------
# 1) Fato principal
# Campos mínimos necessários
# ---------------------------------------------------------
fact_cols = [
    "ra","ano_base",                                          #chaves
    "fase","fase_raw","turma", "instituicao","escola",        #contexto
    "matem","portug","ingles","ieg","iaa","ips","ipp",        #medidas acadêmicas/psicossocais
    "defasagem"                                               #alvo
]
fact_cols = [c for c in fact_cols if c in df_all.columns]
fact_student_year = df_all[fact_cols].copy()

# Dedup: 1 linha por ra+ano_base (se houver duplicatas, mantém a primeira)
fact_student_year = (
    fact_student_year
    .sort_values(["ra","ano_base"])
    .drop_duplicates(subset=["ra","ano_base"], keep="first")
    .reset_index(drop=True)
)


In [79]:
fact_student_year.shape

(3030, 15)

In [80]:
fact_student_year.columns

Index(['ra', 'ano_base', 'fase', 'fase_raw', 'turma', 'instituicao', 'escola',
       'matem', 'portug', 'ingles', 'ieg', 'iaa', 'ips', 'ipp', 'defasagem'],
      dtype='object')

In [81]:
# ---------------------------------------------------------
# 2) DIM: dim_student (1 linha por ra)
# Atributos para identifacção (BI interno)
# ---------------------------------------------------------
student_cols = ["ra","genero","data_nasc","ano_nasc","idade","ano_ingresso","nome"]
student_cols = [c for c in student_cols if c in df_all.columns]

dim_student = df_all[student_cols].copy()

# Preferência: dados "mais recentes" do aluno (maior ano_base)
dim_student = (
    dim_student
    .sort_values(["ra"])
    .drop_duplicates(subset=["ra"], keep="last")
    .reset_index(drop=True)
)

In [82]:
print(dim_student.shape)

(1661, 7)


In [83]:
# ---------------------------------------------------------
# 3) DIM: dim_indicators_year (ra+ano_base)
# indicadores compostos e classificações
# contexto/BI e explicações (não para treino)
# ---------------------------------------------------------
ind_cols = [
    "ra","ano_base",                                        #Chave
    "inde","pedra","ida","ian","ipv",                       #indicadores
    "fase_ideal","class_geral","class_fase","class_turma"   #pós-processado
]
ind_cols = [c for c in ind_cols if c in df_all.columns]

dim_indicators_year = df_all[ind_cols].copy()

dim_indicators_year = (
    dim_indicators_year
    .sort_values(["ra","ano_base"])
    .drop_duplicates(subset=["ra","ano_base"], keep="first")
    .reset_index(drop=True)
)

In [84]:
# ---------------------------------------------------------
# 4) DIM: dim_assessment_process (ra+ano_base)
# Processo de avaliação / auditoria / veriifcar vieses operacionais
# Não entra no modelo
# ---------------------------------------------------------
proc_cols = [
    "ra","ano_base",                                                                  #chave
    "no_av",                                                                          #processo
    "avaliador1","avaliador2","avaliador3","avaliador4","avaliador5","avaliador6",    #Avaliadores
    "rec_av1","rec_av2","rec_av3","rec_av4",                                          #Recomendações
    "rec_psicologia", "indicado","atingiu_pv",                                        #Psicologia/decisões
    "destaque_ieg","destaque_ida","destaque_ipv",                                     #Destaques
    "ativo_inativo"                                                                   #Status
]
proc_cols = [c for c in proc_cols if c in df_all.columns]

dim_assessment_process = df_all[proc_cols].copy()

dim_assessment_process = (
    dim_assessment_process
    .sort_values(["ra","ano_base"])
    .drop_duplicates(subset=["ra","ano_base"], keep="first")
    .reset_index(drop=True)
)

In [85]:
# ---------------------------------------------------------
# 5) VIEWS
# ---------------------------------------------------------

# 5.1 View ML (sem leakage):
# Objetivo: dataset para ML sem leakage
# fact + alguns campos do dim_student
ml_student_fields = [c for c in ["genero","ano_ingresso","ano_nasc","idade"] if c in dim_student.columns]

vw_ml_train_ready_star = (
    fact_student_year
    .merge(dim_student[["ra"] + ml_student_fields], on="ra", how="left")
)

# (Opcional) remover 'defasagem' da lista de features depois na hora de montar X/y

# 5.2 View Dashboard 360:
# Objetivo: visão 360 para os coordenadores
# fact + indicators + processo (campos selecionados)
dash_proc_fields = [c for c in ["no_av","rec_psicologia","ativo_inativo"] if c in dim_assessment_process.columns]
vw_dashboard_360 = (
    fact_student_year
    .merge(dim_indicators_year, on=["ra","ano_base"], how="left", suffixes=("","_ind"))
    .merge(dim_assessment_process[["ra","ano_base"] + dash_proc_fields], on=["ra","ano_base"], how="left")
)

# 5.3 View Auditoria:
# Objetivo: auditoria do processo de avaliação
# processo + fato (para ver relação com alvo e notas)
audit_fact_fields = [c for c in ["defasagem","fase","turma","matem","portug","ieg","ips","iaa"] if c in fact_student_year.columns]
vw_audit_assessment = (
    dim_assessment_process
    .merge(fact_student_year[["ra","ano_base"] + audit_fact_fields], on=["ra","ano_base"], how="left")
)

# ---------------------------------------------------------
# 6) Checks rápidos
# ---------------------------------------------------------
print("fact_student_year:", fact_student_year.shape, "| PK únicos:", fact_student_year[["ra","ano_base"]].drop_duplicates().shape[0])
print("dim_student:", dim_student.shape, "| PK únicos:", dim_student["ra"].nunique())
print("dim_indicators_year:", dim_indicators_year.shape)
print("dim_assessment_process:", dim_assessment_process.shape)
print("vw_ml_train_ready_star:", vw_ml_train_ready_star.shape)
print("vw_dashboard_360:", vw_dashboard_360.shape)
print("vw_audit_assessment:", vw_audit_assessment.shape)

fact_student_year: (3030, 15) | PK únicos: 3030
dim_student: (1661, 7) | PK únicos: 1661
dim_indicators_year: (3030, 11)
dim_assessment_process: (3030, 20)
vw_ml_train_ready_star: (3030, 19)
vw_dashboard_360: (3030, 27)
vw_audit_assessment: (3030, 28)


In [86]:
print("fact_student_year:", fact_student_year.shape, "| PK únicos:", fact_student_year[["ra","ano_base"]].drop_duplicates().shape[0])
print("vw_ml_train_ready_star:", vw_ml_train_ready_star.shape, "| PK únicos:", vw_ml_train_ready_star[["ra","ano_base"]].drop_duplicates().shape[0])

fact_student_year: (3030, 15) | PK únicos: 3030
vw_ml_train_ready_star: (3030, 19) | PK únicos: 3030


In [87]:
df_all['defasagem'].unique()

<IntegerArray>
[-1, 0, -2, -3, 2, 1, -4, -5, 3]
Length: 9, dtype: Int64

In [88]:
df_all['defasagem'].sum()

np.int64(-1948)

In [89]:
df_all[(df_all['ano_base'] == 2022) & (df_all['defasagem'] < 0)]['defasagem']

0      -1
5      -1
6      -1
7      -1
8      -1
       ..
846    -1
847    -1
848    -1
854    -1
856    -1
Name: defasagem, Length: 601, dtype: Int64

In [90]:
df_all[(df_all['ano_base'] == 2022) & (df_all['defasagem'] == 0)]['defasagem']

1      0
2      0
3      0
4      0
11     0
      ..
853    0
855    0
857    0
858    0
859    0
Name: defasagem, Length: 247, dtype: Int64

In [91]:
df_all[(df_all['ano_base'] == 2022) & (df_all['defasagem'] > 0)]['defasagem']

51     2
80     1
89     2
99     1
102    1
110    1
142    1
156    1
160    2
273    1
501    1
600    1
Name: defasagem, dtype: Int64

In [92]:
df_all[(df_all['ano_base'] == 2023) & (df_all['defasagem'] < 0)]['defasagem']

861     -1
869     -1
870     -1
871     -2
872     -2
        ..
1790    -1
1792    -1
1796    -1
1798    -1
1800    -1
Name: defasagem, Length: 552, dtype: Int64

In [93]:
df_all[(df_all['ano_base'] == 2023) & (df_all['defasagem'] == 0)]['defasagem']


860     0
862     0
863     0
864     0
865     0
       ..
1868    0
1869    0
1870    0
1871    0
1873    0
Name: defasagem, Length: 420, dtype: Int64

In [94]:
df_all[(df_all['ano_base'] == 2023) & (df_all['defasagem'] > 0)]['defasagem']

1099    1
1261    1
1472    1
1481    1
1482    1
1558    1
1573    1
1588    1
1598    1
1600    1
1603    1
1605    1
1606    1
1614    1
1618    1
1639    1
1644    1
1648    1
1672    1
1675    1
1682    1
1685    1
1691    1
1700    1
1701    1
1705    1
1724    1
1726    1
1743    2
1747    1
1748    1
1799    2
1801    1
1802    1
1803    2
1804    1
1806    1
1807    2
1808    2
1810    1
1860    1
1872    1
Name: defasagem, dtype: Int64

In [95]:
df_all['fase'].unique()

<IntegerArray>
[7, 6, 5, 4, 3, 2, 1, 0, 8, 9]
Length: 10, dtype: Int64

## 4. Formulação do Problema — Target e Estratégia Longitudinal

**Pergunta central:** quais alunos piorarão de condição educacional no próximo ciclo?

O target é definido como **piora de defasagem** entre dois anos consecutivos:

$$\text{target} = \mathbb{1}[\,\text{defasagem}_{t+1} > \text{defasagem}_{t}\,]$$

Esta formulação cobre todos os cenários relevantes:

| Cenário | `defasagem_t` | `defasagem_t+1` | `target` |
|---|---|---|---|
| Entrada em defasagem | 0 | 1 | **1** |
| Agravamento | 1 | 2 | **1** |
| Estável | 1 | 1 | 0 |
| Melhora | 2 | 1 | 0 |

**Por que não restringir a alunos com `defasagem == 0`?**  
A maioria dos alunos já tem alguma defasagem. Restringir o conjunto aos "em dia" reduziria drasticamente o tamanho do dataset e descartaria o sinal de agravamento — exatamente o caso mais crítico operacionalmente.

> **Todos os alunos com par T→T+1 válido são elegíveis**, independente da defasagem inicial.


### 4.1 Montagem Longitudinal (T → T+1)

Cada aluno que aparece em dois anos consecutivos gera **um par de treino**: as features do ano T e o target derivado do ano T+1.

```
base_t  (features em T)
    ⋈  base_t1  (defasagem de T+1, alinhada pelo RA)
    →  pairs_all  (par T→T+1 com target)
```

O alinhamento é feito subtraindo 1 de `ano_base` no DataFrame T+1 antes do join — assim o INNER JOIN naturalmente descarta alunos sem continuidade no ano seguinte (sem ground truth).


### 4.2 Configuração do Split Temporal

O split respeita a ordem cronológica — nenhum dado futuro é usado para treinar ou validar o passado:

| Partição | Par | N esperado |
|---|---|---|
| **Treino** | 2022 → 2023 | ~alunos com RA nos dois anos |
| **Validação** | 2023 → 2024 | ~alunos com RA nos dois anos |

**Exemplos de pares:**

| `pair_label` | `ra` | `defasagem_t` | `defasagem_t1` | `target` |
|---|---|---|---|---|
| 2022→2023 | A | 0 | 0 | 0 |
| 2022→2023 | B | 0 | 2 | **1** |
| 2023→2024 | A | 0 | 3 | **1** |
| 2023→2024 | D | 0 | 0 | 0 |

> O último par cronológico disponível é sempre reservado para validação. Se novos anos forem adicionados ao PEDE, a lógica de split se adapta automaticamente.


In [96]:
# ---------------------------------------------------------
# 0) Base do ano t (features no tempo t)
# ---------------------------------------------------------
base_t = vw_ml_train_ready_star.copy()

# Garantias mínimas
base_t["ra"] = base_t["ra"].astype(str).str.strip()
base_t["ano_base"] = pd.to_numeric(base_t["ano_base"], errors="coerce").astype("Int64")

# (Opcional, mas recomendado) dedup defensivo: 1 linha por ra+ano_base
base_t = (
    base_t.sort_values(["ra","ano_base"])
          .drop_duplicates(subset=["ra","ano_base"], keep="first")
          .reset_index(drop=True)
)

# ---------------------------------------------------------
# 1) Base do ano t+1 (somente o que vem do futuro para criar o target)
#    Aqui pegamos a defasagem do ano seguinte e "alinhamos" ela no ano t
# ---------------------------------------------------------
base_t1 = base_t[["ra","ano_base","defasagem"]].copy()
base_t1["ano_base"] = base_t1["ano_base"] - 1  # alinha: defasagem(t+1) vira defasagem_t1 no ano t
base_t1 = base_t1.rename(columns={"defasagem": "defasagem_t1"})

# ---------------------------------------------------------
# 2) Join t com t+1 alinhado => gera pares (t -> t+1)
# ---------------------------------------------------------
pairs_all = (
    base_t.merge(base_t1, on=["ra","ano_base"], how="inner")
)

# Renomeia defasagem do ano t
pairs_all = pairs_all.rename(columns={"defasagem": "defasagem_t"})

# Cria label e target (Estratégia 3)
pairs_all["pair_label"] = pairs_all["ano_base"].astype(int).astype(str) + "->" + (pairs_all["ano_base"].astype(int)+1).astype(str)
pairs_all["target"] = (pairs_all["defasagem_t1"] > pairs_all["defasagem_t"]).astype(int)

# ---------------------------------------------------------
# 3) Checks rápidos
# ---------------------------------------------------------
print("pairs_all:", pairs_all.shape)
print("pares disponíveis:", pairs_all["pair_label"].value_counts().sort_index().to_dict())
print("base_rate global (piora):", float(pairs_all["target"].mean()))

# Opcional: base rate por par
print(
    pairs_all.groupby("pair_label")["target"]
             .agg(n="count", n_pos="sum", base_rate="mean")
             .sort_index()
)

pairs_all: (1365, 22)
pares disponíveis: {'2022->2023': 600, '2023->2024': 765}
base_rate global (piora): 0.3648351648351648
              n  n_pos  base_rate
pair_label                       
2022->2023  600    185   0.308333
2023->2024  765    313   0.409150


### 4.3 Separação Treino e Validação

Os pares são ordenados cronologicamente e o último é reservado para validação out-of-time. Todos os pares anteriores compõem o treino.


In [97]:
# Lista de pares ordenados
pairs = [g.copy() for _, g in pairs_all.groupby("pair_label", sort=True)]

# Último par como validação
valid_df = pairs[-1].reset_index(drop=True)
train_df = pd.concat(pairs[:-1], ignore_index=True)

print("TREINO (pares):", [p["pair_label"].iloc[0] for p in pairs[:-1]])
print("VALID (par):", valid_df["pair_label"].unique().tolist())

print("train_df:", train_df.shape, " | base_rate:", float(train_df["target"].mean()))
print("valid_df:", valid_df.shape, " | base_rate:", float(valid_df["target"].mean()))

TREINO (pares): ['2022->2023']
VALID (par): ['2023->2024']
train_df: (600, 22)  | base_rate: 0.30833333333333335
valid_df: (765, 22)  | base_rate: 0.4091503267973856


### 4.4 Montagem de X e y (sem leakage)

Separa features (`X`) do target (`y`), excluindo explicitamente toda informação do futuro:

| Coluna | Papel | Incluída em X? |
|---|---|---|
| `defasagem_t` | Feature — defasagem no ano T | ✅ Sim |
| `defasagem_t1` | Ground truth do futuro (T+1) | ❌ Nunca |
| `target` | Label supervisionado | ❌ Nunca |
| `pair_label` | Identificador do par | ❌ Metadado |
| `ra`, `ano_base` | Chaves de identificação | ❌ Não são features |


In [98]:
leak_cols = ["target", "defasagem_t1", "pair_label"]  # e qualquer coluna do futuro se existir
id_cols = ["ra", "ano_base"]

feature_cols = [c for c in train_df.columns if c not in leak_cols + id_cols]

X_train = train_df[feature_cols]
y_train = train_df["target"].astype(int)

X_valid = valid_df[feature_cols]
y_valid = valid_df["target"].astype(int)

print("N features:", len(feature_cols))

N features: 17


### 4.5 Sanity Checks

Verificações mínimas antes de exportar os datasets:

- **Variabilidade do target** — treino e validação devem ter exemplos de ambas as classes (0 e 1)
- **Unicidade da chave** — nenhum par `ra + ano_base` duplicado em `pairs_all`


In [99]:
# 1) se o target é sempre 0 ou sempre 1, o modelo não aprende
assert train_df["target"].nunique() == 2, "Treino sem variabilidade no target."
assert valid_df["target"].nunique() == 2, "Validação sem variabilidade no target."

# 2) checar se tem duplicata de ra+ano_base no dataset pareado
assert not pairs_all.duplicated(subset=["ra","ano_base"]).any(), "Duplicata em ra+ano_base dentro de pairs_all (não deveria)."

In [100]:
valid_df

,ra,ano_base,fase,fase_raw,turma,instituicao,escola,matem,portug,ingles,...,ips,ipp,defasagem_t,genero,ano_ingresso,ano_nasc,idade,defasagem_t1,pair_label,target
0,RA-1,2023,8,FASE 8,8E,Privada *Parcerias com Bolsa 100%,NaN,NaN,NaN,NaN,...,NaN,NaN,0,F,2021,2003,21,0,2023->2024,0
1,RA-1000,2023,0,ALFA,ALFA U - G2/G3,Pública,NaN,7.0,7.0,NaN,...,3.77,6.250,0,F,2023,2015,8,0,2023->2024,0
2,RA-1001,2023,0,ALFA,ALFA U - G2/G3,Pública,NaN,8.3,7.3,NaN,...,7.52,7.500,-1,F,2023,2014,10,-1,2023->2024,0
3,RA-1002,2023,0,ALFA,ALFA U - G2/G3,Pública,NaN,7.0,7.0,NaN,...,7.52,6.250,-1,F,2023,2014,9,-1,2023->2024,0
4,RA-1003,2023,0,ALFA,ALFA V - G0/G1,Pública,NaN,8.0,0.0,NaN,...,7.52,7.500,0,M,2023,2015,8,-1,2023->2024,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,RA-989,2023,0,ALFA,ALFA S - G2/G3,Pública,NaN,8.7,7.8,NaN,...,2.52,7.500,-1,M,2023,2014,10,0,2023->2024,1
761,RA-990,2023,0,ALFA,ALFA T - G2/G3,Pública,NaN,8.3,7.2,NaN,...,7.52,7.500,-1,M,2023,2014,10,-1,2023->2024,0
762,RA-991,2023,0,ALFA,ALFA T - G2/G3,Pública,NaN,8.0,7.3,NaN,...,3.14,6.875,-1,F,2023,2014,10,-1,2023->2024,0
763,RA-994,2023,0,ALFA,ALFA T - G2/G3,Pública,NaN,6.2,5.7,NaN,...,3.77,5.625,0,F,2023,2014,10,-1,2023->2024,0


## 5. Exportação — Parquets de Treino e Validação

Persiste os datasets finais em formato Parquet para consumo pela etapa de engenharia de features:

| Arquivo | Conteúdo |
|---|---|
| `notebooks/train_df.parquet` | Pares 2022→2023 com features e target |
| `notebooks/valid_df.parquet` | Par 2023→2024 com features e target |


In [101]:
DATA_DIR = NOTEBOOKS_DIR / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)

train_df.to_parquet(DATA_DIR / "train_df.parquet", index=False)
valid_df.to_parquet(DATA_DIR / "valid_df.parquet", index=False)

print("✅ Parquets salvos em:", DATA_DIR)
print(" - train_df.parquet:", (DATA_DIR / "train_df.parquet").stat().st_size, "bytes")
print(" - valid_df.parquet:", (DATA_DIR / "valid_df.parquet").stat().st_size, "bytes")


✅ Parquets salvos em: /home/glauberthy/Desktop/datathon/notebooks/data
 - train_df.parquet: 22170 bytes
 - valid_df.parquet: 26300 bytes
